In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import glob
import math


import re, string
import seaborn as sns
import pickle


from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')

from src.data.plot_functions import *
from src.data.feature_engineering import preprocess_train_df, tokenize
from src.conf.global_variables import TOXIC_COMMENT_DATA_PATH, SEED, LABELS, COMMENT
from src.models.train_model import fit_models
from src.models.predict_model import get_prediction
from src.models.save_and_load_models import load_models

%load_ext autoreload
%autoreload 2

## Load the Model 

In [2]:
models_filenames = glob.glob('./models/model_*.pkl')
vectorizer_filename = './models/vectorizer.pkl'

In [3]:
classifiers, vectorizer = load_models(models_filenames, vectorizer_filename)

In [4]:
classifiers

{'toxic': LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'obscene': LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'threat': LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
             

## Prediction

In [5]:
sentence = 'hey fucking bitch'

In [6]:
test_term_doc = vectorizer.transform(pd.Series(sentence))

In [7]:
get_prediction(test_term_doc, classifiers, LABELS)

array([[0.99262731, 0.10188223, 0.99489038, 0.01314715, 0.97547554,
        0.0301334 ]])

## Interpret prediction

In [8]:
classifiers['toxic'].coef_

array([[ 0.01639266, -0.03434133,  0.22021969, ...,  0.0201806 ,
        -0.00066565, -0.02979101]])

In [9]:
classifiers['toxic'].intercept_

array([-2.3794149])

Get only the 3 coefficients

In [10]:
test_term_doc.indices

array([ 6712, 19572, 22417], dtype=int32)

In [11]:
words = vectorizer.get_feature_names()

In [12]:
classifiers['toxic'].coef_.ravel()[test_term_doc.indices]

array([2.67368732, 4.57070167, 0.03759862])

In [13]:
np.array(words)[test_term_doc.indices]

array(['bitch', 'fucking', 'hey'], dtype='<U31')

Créons maintenant un dataframe propre avec les mots / poids

In [14]:
df_words_weights = pd.DataFrame({'words': np.array(words)[test_term_doc.indices],
                                 'weights': classifiers['toxic'].coef_.ravel()[test_term_doc.indices]})

In [15]:
df_words_weights

,words,weights
0,bitch,2.673687
1,fucking,4.570702
2,hey,0.037599


On peut d'abord recalculer la proba pour être sûr

In [16]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [24]:
x = classifiers['toxic'].intercept_.ravel() + df_words_weights['weights'].sum()

In [18]:
proba = sigmoid(x)
print(proba)

0.9926273104718545


In [22]:
def odds(x):
    return sigmoid(x) / (1 - sigmoid(x))

In [29]:
odds_with_feature = odds(x)
print(odds_with_feature)

134.63571288095926


Comment on interprète ça ?

Si on enlève fucking, 

In [30]:
sentence = 'hey bitch'

In [31]:
test_term_doc = vectorizer.transform(pd.Series(sentence))

In [32]:
get_prediction(test_term_doc, classifiers, LABELS)

array([[0.58221456, 0.04304962, 0.5308347 , 0.00830071, 0.57172791,
        0.01286596]])

In [35]:
proba_without_feature = get_prediction(test_term_doc, classifiers, LABELS)[0][0]

In [36]:
proba_without_feature

0.5822145603574197

In [37]:
odds_without_feature = odds(proba_without_feature)
print(odds_without_feature)

1.789998103504518


In [47]:
def odds_withdraw_feature(odds_with_feature, weight_feature):
    return odds_with_feature / np.exp(weight_feature)

In [52]:
odds_withdraw_feature(odds_with_feature, 4.570702)

1.3935726591070319